In [1]:
import tensorflow as tf
import scipy.misc
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.io as sio
from model import *
import datetime
from read_utils import * # Import read_images and aug_images
%matplotlib inline


C:\Users\yarki\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
tf.__version__
#Check if the version is >= 1.4

'1.4.0'

In [3]:
DATA_PATH = 'data\\realwaves_outputs' #Real Ocean Textures
NORMAL_PATH = 'data\\normalmap_outputs' #Ocean Normals Textures
LABEL_PATH = 'data\\styled_outputs' #Groudn Truth Texture Renders
AUG_DATA_PATH ='data\\aug_realwaves_outputs'
AUG_LABEL_PATH ='data\\aug_styled_outputs'
AUG_NORMAL_PATH = 'data\\aug_normalmap_outputs'

TEST_NORMAL_PATH = 'data\\test\\normal_2' #Ocean Normals Textures

TEST_OUTPUT_DIR = 'generated'

AUGMENT = True
if AUGMENT:
    aug_images(DATA_PATH, AUG_DATA_PATH, save=False)
    aug_images(NORMAL_PATH, AUG_NORMAL_PATH, save=False)
    aug_images(LABEL_PATH, AUG_LABEL_PATH, save=False)


#label_images = read_images(LABEL_PATH)

KeyboardInterrupt: 

In [ ]:
## HYPERPARAMETERS
# DATASET PARAMETERS
im_size = 128
n_epochs = 1000
batch_size = 16

BLOCKS = 4 #Number of residual blocks

START_LR = 1e-4 #Starting learning rate
VGG_COEFF = 5e-5
GAN_COEFF = 5e-2


In [ ]:
def image_gen():
    for filename1, filename2, filename3 in zip(os.listdir(AUG_DATA_PATH),os.listdir(AUG_NORMAL_PATH), os.listdir(AUG_LABEL_PATH)):
        yield (np.array(sio.loadmat(os.path.join(AUG_DATA_PATH, filename1))['img_raw'])), 
        (np.array(sio.loadmat(os.path.join(AUG_NORMAL_PATH, filename2))['img_raw'])), (np.array(sio.loadmat(os.path.join(AUG_LABEL_PATH, filename3))['img_raw']))

with tf.variable_scope('dataset'):
    dset = tf.data.Dataset.from_generator(image_gen, (tf.float32, tf.float32, tf.float32), (tf.TensorShape([im_size,im_size,3]), tf.TensorShape([im_size,im_size,3]),tf.TensorShape([im_size,im_size,3])))
    dset = dset.shuffle(1000)
    dset = dset.repeat(n_epochs)
    dset = dset.batch(batch_size)
    dset = dset.prefetch(4)
    dset_iterator = dset.make_initializable_iterator()
    next_batch = dset_iterator.get_next()

In [ ]:
def Stylizer_Resnet(input_net, scope, REUSE):
    BLOCKS = 8
    #pad = BLOCKS * 2 * 8
    #net = tf.pad(input_net, [[0,0], [pad,pad], [pad,pad], [0,0]], "REFLECT") 
    with tf.variable_scope(scope):
        net = tf.layers.conv2d(input_net, 16, [3,3],padding='SAME', reuse=REUSE, name='conv0')
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn0')
        net = tf.nn.relu(net)
        net = tf.layers.conv2d(net, 32, [3,3],padding='SAME', strides=[2,2], reuse=REUSE, name='conv1')
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn1')
        net = tf.nn.relu(net)
        net = tf.layers.conv2d(net, 64, [3,3],padding='SAME', strides=[2,2], reuse=REUSE, name='conv2')
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn2')
        net = tf.nn.relu(net)
        #net = tf.layers.conv2d(net, 64, [5,5],padding='SAME', strides=[2,2], activation=tf.nn.relu, reuse=REUSE, name='conv1')
        for i in range(BLOCKS):
            net = ResidualBlock(net, i, REUSE)
        #net = tf.layers.conv2d(net, 64, [5,5],padding='SAME', reuse=REUSE, name='conv5')
        #net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn6')
        #net = tf.nn.relu(net)
        #net = tf.image.resize_images(net, size=[im_size,im_size], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        net = tf.layers.conv2d_transpose(net, 64, [3,3], strides=[2,2], padding='SAME', reuse=REUSE, name='deconv1')
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn5')
        net = tf.nn.relu(net)
        net = tf.layers.conv2d_transpose(net, 32, [3,3],strides=[2,2], padding='SAME', reuse=REUSE, name='deconv2')
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn6')
        net = tf.nn.relu(net)
        net = tf.layers.conv2d(net, 16, [3,3],padding='SAME', reuse=REUSE, name='conv6')
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='bn7')
        net = tf.nn.relu(net)
        net = tf.layers.conv2d(net, 3, [3,3],padding='SAME', reuse=REUSE, name='conv7')
        net = tf.nn.tanh(net)
        return net
        
def Discriminator(net, scope, REUSE):
    
    with tf.variable_scope(scope):
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn0')
        net = tf.layers.conv2d(net, 32, [5,5], strides=[1,1],padding='SAME', reuse=REUSE, name='d_conv1')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn1')
        
        net = tf.layers.conv2d(net, 32, [5,5], strides=[2,2],padding='SAME', reuse=REUSE, name='d_conv1_2')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn1_2')
        
        
        
        net = tf.layers.conv2d(net, 64, [5,5],strides=[1,1],padding='SAME', reuse=REUSE, name='d_conv2')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn2')
        
        net = tf.layers.conv2d(net, 64, [5,5],strides=[2,2],padding='SAME', reuse=REUSE, name='d_conv2_2')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn2_2')
        
        
        
        net = tf.layers.conv2d(net, 128, [5,5], strides=[1,1],padding='SAME', reuse=REUSE, name='d_conv3')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn3')
        
        net = tf.layers.conv2d(net, 128, [5,5], strides=[2,2],padding='SAME', reuse=REUSE, name='d_conv3_2')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn3_2')
        
        
        
        net = tf.layers.conv2d(net, 256, [5,5], strides=[1,1],padding='SAME', reuse=REUSE, name='d_conv4')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn4')
        
        net = tf.layers.conv2d(net, 256, [5,5], strides=[2,2],padding='SAME', reuse=REUSE, name='d_conv4_2')
        net = lrelu(net, 0.2)
        net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn4_2')
        
        net = tf.layers.conv2d(net, 256, [3,3], strides=[2,2],padding='SAME', reuse=REUSE, name='d_conv5')
        #net = lrelu(net, 0.2)
        #net = tf.layers.batch_normalization(net, reuse=REUSE, name='d_bn5')

        net = tf.contrib.layers.flatten(net)
        net = tf.layers.dense(net, 128, activation=tf.nn.relu, reuse=REUSE, name='d_dense1')
        net = tf.layers.dense(net, 1, reuse=REUSE, name='d_dense2')
        
    return net
    
def lrelu(x, alpha):
    return tf.nn.relu(x) - alpha * tf.nn.relu(-x)
    
def ResidualBlock(net, id, REUSE):
    #_, rows, cols, channels = [i.value for i in net.get_shape()]
    layer_1 = tf.layers.conv2d(net, 64, [5,5],padding='SAME', reuse=REUSE, name='res_conv%d_1' % id)
    bn_1 = tf.nn.relu(tf.layers.batch_normalization(layer_1, reuse=REUSE, name='res_bn%d_1' % id))
    
    layer_2 = tf.layers.conv2d(bn_1, 64, [5,5],padding='SAME', reuse=REUSE, name='res_conv%d_2' % id)  #tf.slice(net,[0,2,2,0], [batch_size,rows-4,cols-4,channels])
    bn_2 = tf.layers.batch_normalization(layer_2, reuse=REUSE, name='res_bn%d_2' % id) +  net 
    return bn_2

In [ ]:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = START_LR
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.9, staircase=True)
data = next_batch[0]
normals = next_batch[1]
labels = next_batch[2]

#Form the generator
predictions = Stylizer_Resnet(normals, 'generator', False)

#VGG inputs for ground truth and generated images
t_target_image_224 = tf.image.resize_images(labels, size=[224, 224], method=0, align_corners=False) # resize_target_image_for_vgg # http://tensorlayer.readthedocs.io/en/latest/_modules/tensorlayer/layers.html#UpSampling2dLayer
t_predict_image_224 = tf.image.resize_images(predictions, size=[224, 224], method=0, align_corners=False) # resize_generate_image_for_vgg

#VGG outputs for ground truth and generated images
_, vgg_target_emb = Vgg19_simple_api((t_target_image_224+1)/2, reuse=False)
_, vgg_predict_emb = Vgg19_simple_api((t_predict_image_224+1)/2, reuse=True)
    
#Form the discriminator for real and fake images
d_real_output = Discriminator(data, 'discriminator', False)
d_fake_output = Discriminator(predictions, 'discriminator', True)
    
#Collect the generator and discriminator variables into seperate collections
g_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
d_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')

r_labels = tf.ones(dtype='float32', shape=[batch_size,1]) + tf.random_uniform(minval=-1.0, shape=[batch_size,1]) * 0.2
f_labels = tf.zeros(dtype='float32', shape=[batch_size,1]) + tf.random_uniform(minval=0.0, shape=[batch_size,1]) * 0.2

print("Model is built.")


In [ ]:
def sigmoid_cross_entropy_with_logits(x, y):
      try:
        return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, labels=y)
      except:
        return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, targets=y)

#Form VGG and MSE losses.
vgg_loss = VGG_COEFF * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)
mse_loss = tl.cost.mean_squared_error(predictions , labels, is_mean=True)


#Form GAN losses
d_loss_real = tf.reduce_mean(sigmoid_cross_entropy_with_logits(d_real_output, r_labels))
d_loss_fake = tf.reduce_mean(sigmoid_cross_entropy_with_logits(d_fake_output, f_labels))
d_loss = d_loss_real + d_loss_fake
g_gan_loss = GAN_COEFF *  tf.reduce_mean(sigmoid_cross_entropy_with_logits(d_fake_output, r_labels))


g_loss = mse_loss + vgg_loss + g_gan_loss

#Form train ops
g_train = tf.train.AdamOptimizer(learning_rate,beta1=0.5, beta2= 0.999).minimize(g_loss, var_list=g_vars)
vgg_train = tf.train.AdamOptimizer(learning_rate,beta1=0.5, beta2= 0.999).minimize(g_loss, var_list=g_vars)
d_train = tf.train.AdamOptimizer(learning_rate,beta1=0.5, beta2= 0.999).minimize(d_loss, var_list=d_vars)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(dset_iterator.initializer)
saver = tf.train.Saver()
#saver.restore(sess,'models/model4.ckpt') #Uncomment to load model from ckpt

In [ ]:
#Load the VGG model courtesy of machrisaa

vgg19_npy_path = "vgg19.npy"

if not os.path.isfile(vgg19_npy_path):

    print("Please download vgg19.npz from : https://github.com/machrisaa/tensorflow-vgg")
else:

    npz = np.load(vgg19_npy_path, encoding='latin1').item()
    params = []

    for val in sorted( npz.items() ):

        W = np.asarray(val[1][0])

        b = np.asarray(val[1][1])

        print("  Loading %s: %s, %s" % (val[0], W.shape, b.shape))

        params.extend([W, b])

    tl.files.assign_params(sess, params, net_vgg)

In [ ]:
sess.run(dset_iterator.initializer)
i = 0
while True:
    try:
        i += 1
        _,d_l = sess.run([d_train, d_loss])
        _,g_l = sess.run([g_train, vgg_loss])
        mse_l = sess.run(mse_loss)
        #d_l = 0
        if i % 10 == 0:
            print('Loss for iteration %d D: %f G: %f MSE: %f' % (i, d_l, g_l, mse_l))
        if np.isnan(mse_l):
            break
        if i % 100 == 0:
            if not np.isnan(mse_l):
                path = os.path("models", "MODEL_NAME" + ".ckpt")
                save_path = saver.save(sess, path)
    except tf.errors.OutOfRangeError:
        print("End of dataset")  # ==> "End of dataset"
        sess.run(dset_iterator.initializer)
        break

In [ ]:
output_op = (predictions, normals, labels, data)
A = sess.run(output_op)
print(A[0].shape)
print(np.max(A[0]))
im3 = A[3][0]/2+0.5
im0 = A[0][0]/2+0.5
im1 = A[1][0]/2+0.5
im2 = A[2][0]/2+0.5
print(im1.shape)
print(np.max(A[2]))
print(np.min(A[2]))
plot = plt.figure(1)
plt.subplot(141)
plt.imshow(im3)
plt.subplot(142)
plt.imshow(im1)
plt.subplot(143)
plt.imshow(im0)
plt.subplot(144)
plt.imshow(im2, vmin=-1.0, vmax=1.0)

In [ ]:
test_normal_input = tf.placeholder(dtype='float32', shape=[1,1024,1024,3])
test_prediction = Stylizer_Resnet(test_normal_input, 'generator', REUSE=True)

In [ ]:
test_normal_ims = read_images(TEST_NORMAL_PATH)[:,:,:,0:3]
train_normal_ins = read_images(NORMAL_PATH)[:,:,:,0:3]
train_label_ins = read_images(LABEL_PATH)[:,:,:,0:3]

In [ ]:
for k in range(250):

    print(test_normal_ims[k:k+1][0].shape)
    test_normal = test_normal_ims[k:k+1] if k==0 else (test_normal_ims[k-1:k] + test_normal_ims[k:k+1])/2
    test_normal = test_normal-np.min(test_normal)
    test_normal = test_normal/np.max(test_normal)
    modifier = 0.45
    test_normal =(test_normal-modifier)*2
    
    start = datetime.datetime.now()
    test_out = sess.run(test_prediction, feed_dict={test_normal_input:test_normal })
    end = datetime.datetime.now() - start
    test_out = test_out/2+0.5
    scipy.misc.imsave(TEST_OUTPUT_PATH+'/%0.3d.jpg' % k, test_out.squeeze())
    
    print(end,k)
k = 0
print(test_out.shape)
plot = plt.figure(1)
plt.subplot(142)
plt.imshow(test_normal.squeeze()/2 + 0.5)
plt.subplot(143)
plt.imshow(test_out.squeeze())
plt.subplot(144)
plt.imshow(train_label_ins[k])
plt.figure(2)
plt.imshow(test_out.squeeze())
plt.figure(3)
plt.imshow(test_normal.squeeze()/2.5 + 0.5)

    